# Which LLM Wrote This? ChatGPT, Claude, Gemini, or Grok?

Detect which AI model (ChatGPT, Claude, Grok, Gemini, or DeepSeek) generated a piece of text.

**Important:** Make sure to enable GPU runtime!

**For Google Colab:**
- Runtime → Change runtime type → Hardware accelerator → GPU (T4)

**For Kaggle Notebooks:**
- Settings → Accelerator → GPU P100

## Step 1: Install Dependencies

Installing compatible versions of all required packages...

**Note:** This project uses Llama 3, which requires Hugging Face authentication.

In [ ]:
# Install compatible versions to avoid dependency conflicts
!pip install -q transformers==4.46.3 peft==0.13.2 huggingface-hub accelerate
!pip install -q llm2vec==0.2.3

print("✓ Dependencies installed successfully!")

## Step 2: Authenticate with Hugging Face

This model requires Hugging Face authentication. The notebook will try to use secrets from Colab or Kaggle.

In [ ]:
from huggingface_hub import login

# Try to get token from secrets (works on Colab and Kaggle)
try:
    # Try Colab secrets first
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("✓ Successfully authenticated with Hugging Face!")
except:
    try:
        # Try Kaggle secrets
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
        login(token=HF_TOKEN)
        print("✓ Successfully authenticated with Hugging Face!")
    except:
        # If both fail, show instructions
        print("⚠️  Could not find HF_TOKEN in secrets.")
        print("\n📝 Setup Instructions:")
        print("\n1. Request Llama 3 access: https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct")
        print("2. Create token: https://huggingface.co/settings/tokens")
        print("\n3. Add to secrets:")
        print("   • Colab: Click 🔑 icon → Add secret 'HF_TOKEN' → Enable notebook access")
        print("   • Kaggle: Settings → Add-ons → Secrets → Add 'HF_TOKEN'")
        print("\n4. Restart runtime and re-run this cell")
        print("\n⚙️  Alternative - Manual login:")
        from getpass import getpass
        token = getpass("Enter your HuggingFace token: ")
        login(token=token)
        print("✓ Authenticated!")

## Step 3: Download the Classifier

This will download ~16GB. It may take 5-10 minutes.

In [ ]:
!huggingface-cli download Yida/classifier_chat --local-dir ./classifier_chat

## Step 4: Load the Classifier

Define and load the classifier model with memory optimization.

In [ ]:
import torch
import numpy as np
from transformers import AutoConfig, AutoModel, AutoTokenizer
from peft import PeftModel
from llm2vec import LLM2Vec
import gc
import os

def load_classifier(checkpoint_path="./classifier_chat", num_labels=5):
    """Load the pre-trained LLM2Vec classifier with memory optimization."""
    print("Loading classifier with memory optimization...")
    
    # Clear cache first
    torch.cuda.empty_cache()
    gc.collect()
    
    base_model_name = "McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp"
    
    config = AutoConfig.from_pretrained(
        base_model_name,
        trust_remote_code=True,
    )
    
    # Strategy: Try single GPU first, fall back to multi-GPU if OOM
    num_gpus = torch.cuda.device_count()
    print(f"Detected {num_gpus} GPU(s)")
    
    # Always use single GPU - multi-GPU causes device mismatch with residual connections
    # Even with 2 GPUs available, using just one with CPU offload is more reliable
    print("Using single GPU (GPU 0) with CPU offload for stability")
    device_map = {"": 0}  # Force everything to GPU 0
    max_memory = {0: "14GiB", "cpu": "30GiB"}
    
    # Load base model
    print("Loading base model...")
    model = AutoModel.from_pretrained(
        base_model_name,
        config=config,
        torch_dtype=torch.bfloat16,
        device_map=device_map,
        max_memory=max_memory,
        offload_folder="./offload",
        trust_remote_code=True,
        low_cpu_mem_usage=True,
    )
    
    print("✓ Base model loaded")
    torch.cuda.empty_cache()
    
    # Load and merge first adapter
    print("Loading first adapter...")
    model = PeftModel.from_pretrained(
        model,
        base_model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
    )
    
    # Move to CPU for merging to avoid OOM
    print("✓ Moving to CPU for merging...")
    model = model.cpu()
    torch.cuda.empty_cache()
    gc.collect()
    
    print("✓ Merging first adapter...")
    model = model.merge_and_unload()
    
    # Move back to GPU 0 with CPU offload
    print("✓ Moving merged model back to GPU...")
    model = model.to(torch.bfloat16)
    
    # Re-dispatch to GPU 0 (with CPU offload for layers that don't fit)
    from accelerate import dispatch_model, infer_auto_device_map
    device_map_merged = infer_auto_device_map(
        model,
        max_memory=max_memory,
        no_split_module_classes=["LlamaDecoderLayer"]  # Keep decoder layers intact
    )
    model = dispatch_model(model, device_map=device_map_merged, offload_dir="./offload")
    
    torch.cuda.empty_cache()
    gc.collect()
    
    # Load supervised adapter
    print("✓ Loading supervised adapter...")
    model = PeftModel.from_pretrained(
        model,
        f"{base_model_name}-supervised",
        is_trainable=True,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
    )
    
    torch.cuda.empty_cache()
    
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
    model = LLM2Vec(model, tokenizer, pooling_mode="mean", max_length=512)
    
    hidden_size = list(model.modules())[-1].weight.shape[0]
    model.head = torch.nn.Linear(hidden_size, num_labels, dtype=torch.bfloat16)
    
    head_path = os.path.join(checkpoint_path, "head.pt")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.head.load_state_dict(torch.load(head_path, map_location=device))
    model.head = model.head.to(device)
    
    model.eval()
    
    # Final cleanup
    torch.cuda.empty_cache()
    gc.collect()
    
    # Show memory usage across all GPUs
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            reserved = torch.cuda.memory_reserved(i) / 1024**3
            if i == 0 or allocated > 0:
                print(f"✓ GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
    else:
        print("✓ Classifier loaded on CPU")
    
    return model

In [ ]:
# Load the model (this may take a few minutes)
print("Loading model...")
model = load_classifier()
print("✓ Model loaded successfully!")

## Step 5: Define Prediction Function


In [ ]:
def predict_text(model, text):
    """Predict which LLM generated the given text."""
    label_names = ["ChatGPT", "Claude", "Grok", "Gemini", "DeepSeek"]
    
    # Prepare text
    prepared_text = model.prepare_for_tokenization(text)
    inputs = model.tokenize([prepared_text])
    
    # IMPORTANT: For multi-GPU setups, always put inputs on cuda:0
    # The model will handle moving tensors between GPUs automatically
    if torch.cuda.is_available():
        target_device = torch.device("cuda:0")
    else:
        target_device = next(model.parameters()).device
    
    inputs = {k: v.to(target_device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
    
    # Predict
    with torch.no_grad():
        embeddings = model.forward(inputs)
        
        # Move embeddings to same device as classification head
        if hasattr(model, 'head'):
            head_device = next(model.head.parameters()).device
            embeddings = embeddings.to(head_device)
        
        embeddings = embeddings.to(torch.bfloat16)
        logits = model.head(embeddings)
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
    
    pred_label = torch.argmax(probabilities, dim=-1).item()
    # Convert to float32 before numpy (bfloat16 not supported by numpy)
    all_probs = probabilities[0].float().cpu().numpy()
    
    # Display results
    print("\n" + "="*60)
    print("PREDICTION RESULTS")
    print("="*60)
    print(f"\nMost likely source: {label_names[pred_label]}")
    print(f"Confidence: {all_probs[pred_label]*100:.2f}%")
    print("\nAll probabilities:")
    print("-"*60)
    
    sorted_indices = np.argsort(all_probs)[::-1]
    for idx in sorted_indices:
        bar_length = int(all_probs[idx] * 50)
        bar = "█" * bar_length
        print(f"{label_names[idx]:20s} {all_probs[idx]*100:6.2f}% {bar}")
    print("="*60)
    
    return label_names[pred_label], all_probs[pred_label]

## Step 6: Test with Sample Text

Replace the text below with any text you want to classify:

In [ ]:
# Example text - replace with your own!
sample_text = """
Hello! I'd be happy to help you with that question. Let me break this down into a few key points:

1. First, it's important to understand the context
2. Second, we should consider the implications
3. Finally, let's look at practical applications

I hope this helps clarify things for you!
"""

predict_text(model, sample_text)

## Step 7: Classify Your Own Text (Interactive)

Paste your text in the input box below:

In [ ]:
# Interactive input
print("Paste your text below and press Enter:")
user_text = input()

if user_text.strip():
    predict_text(model, user_text)
else:
    print("No text provided!")

## Step 8: Batch Classification (Multiple Texts)

You can test multiple texts at once:

In [ ]:
# Test multiple texts
texts_to_test = [
    "Sure, I can help with that!",
    "I'd be happy to assist you with this question.",
    "Let me break this down for you step by step.",
]

for i, text in enumerate(texts_to_test, 1):
    print(f"\n{'='*60}")
    print(f"TEXT #{i}: {text[:50]}...")
    predict_text(model, text)

## Step 9: Interactive Gradio UI

Launch an interactive web interface to classify text:

In [ ]:
!pip install -q gradio plotly

import gradio as gr
import plotly.graph_objects as go
import io

def predict_gradio(text):
    """Predict for Gradio interface with detailed logs."""
    if not text.strip():
        return "Enter text to analyze", None, "⚠️ No text provided"
    
    log_capture = io.StringIO()
    
    try:
        label_names = ["ChatGPT", "Claude", "Grok", "Gemini", "DeepSeek"]
        
        log_capture.write("🔄 Starting prediction...\n")
        log_capture.write(f"📝 Text length: {len(text)} characters\n")
        
        log_capture.write("\n🔤 Tokenizing input...\n")
        prepared_text = model.prepare_for_tokenization(text)
        inputs = model.tokenize([prepared_text])
        log_capture.write("✓ Tokenization complete\n")
        
        if torch.cuda.is_available():
            target_device = torch.device("cuda:0")
            log_capture.write("\n🖥️  Device: GPU (cuda:0)\n")
        else:
            target_device = next(model.parameters()).device
            log_capture.write(f"\n🖥️  Device: {target_device}\n")
        
        inputs = {k: v.to(target_device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
        
        log_capture.write("\n🧠 Running model inference...\n")
        with torch.no_grad():
            embeddings = model.forward(inputs)
            log_capture.write(f"✓ Generated embeddings: {embeddings.shape}\n")
            
            if hasattr(model, 'head'):
                head_device = next(model.head.parameters()).device
                embeddings = embeddings.to(head_device)
            
            embeddings = embeddings.to(torch.bfloat16)
            logits = model.head(embeddings)
            probabilities = torch.nn.functional.softmax(logits, dim=-1)
            log_capture.write("✓ Computed probabilities\n")
        
        pred_label = torch.argmax(probabilities, dim=-1).item()
        all_probs = probabilities[0].float().cpu().numpy()
        
        log_capture.write(f"\n{'='*40}\n")
        log_capture.write(f"🎯 Prediction: {label_names[pred_label]}\n")
        log_capture.write(f"💯 Confidence: {all_probs[pred_label]*100:.1f}%\n")
        log_capture.write(f"{'='*40}\n\n")
        
        sorted_indices = np.argsort(all_probs)[::-1]
        log_capture.write("📊 All probabilities:\n")
        for idx in sorted_indices:
            bar = "█" * int(all_probs[idx] * 30)
            log_capture.write(f"  {label_names[idx]:12} {all_probs[idx]*100:5.1f}% {bar}\n")
        
        log_capture.write("\n✅ Analysis complete!\n")
        
        # Result text with clear formatting
        result_text = f"## Detected LLM: **{label_names[pred_label]}**\n\n### Confidence: **{all_probs[pred_label]*100:.1f}%**"
        
        # Bar chart
        sorted_labels = [label_names[i] for i in sorted_indices]
        sorted_probs = [float(all_probs[i]) for i in sorted_indices]
        
        colors = ['#1f77b4' if i == 0 else '#aec7e8' for i in range(len(sorted_labels))]
        
        fig = go.Figure(data=[
            go.Bar(
                x=sorted_labels,
                y=sorted_probs,
                text=[f'{p*100:.1f}%' for p in sorted_probs],
                textposition='outside',
                marker_color=colors,
                marker_line_width=0,
            )
        ])
        
        fig.update_layout(
            xaxis_title=None,
            yaxis_title=None,
            yaxis=dict(range=[0, max(sorted_probs) * 1.15], showticklabels=False, showgrid=False),
            xaxis=dict(showgrid=False),
            height=200,
            margin=dict(l=10, r=10, t=10, b=30),
            showlegend=False,
            plot_bgcolor='white',
            paper_bgcolor='white',
        )
        
        return result_text, fig, log_capture.getvalue()
        
    except Exception as e:
        import traceback
        error_msg = f"❌ Error: {str(e)}\n\n{traceback.format_exc()}"
        log_capture.write(error_msg)
        
        empty_fig = go.Figure()
        empty_fig.update_layout(height=200)
        return f"Error: {str(e)}", empty_fig, log_capture.getvalue()


# Single viewport UI with logs
with gr.Blocks(title="Which LLM Wrote This? ChatGPT, Claude, Gemini, or Grok?") as demo:
    gr.Markdown("# Which LLM Wrote This? ChatGPT, Claude, Gemini, or Grok?")
    gr.Markdown("**[Research Paper](https://eric-mingjie.github.io/llm-idiosyncrasies/index.html)** (97% accuracy) • **[GitHub](https://github.com/syedamaann/llm-idiosyncrasies)** • **[syedamaan.com](https://syedamaan.com)**")
    
    with gr.Row():
        # Left: Input
        with gr.Column(scale=1):
            text_input = gr.Textbox(
                label="Input Text",
                placeholder="Paste text here...",
                lines=8,
                max_lines=8,
            )
            submit_btn = gr.Button("Analyze", variant="primary", size="lg")
        
        # Right: Results and Chart
        with gr.Column(scale=1):
            result_output = gr.Markdown(value="**Results will appear here**")
            plot_output = gr.Plot()
    
    # Bottom: Processing logs (compact)
    logs_output = gr.Textbox(
        label="Processing Log",
        lines=8,
        max_lines=8,
        interactive=False,
        show_copy_button=True,
    )
    
    submit_btn.click(
        fn=predict_gradio,
        inputs=text_input,
        outputs=[result_output, plot_output, logs_output]
    )
    
    text_input.submit(
        fn=predict_gradio,
        inputs=text_input,
        outputs=[result_output, plot_output, logs_output]
    )

demo.launch(share=True, debug=True)